In [1]:
import pyscf
from pyscf import gto, dft, scf

Small system for tests:

In [2]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

h2_system = "H 0 0 0; H 1.4 0 0"

n2_system = "N 0 0 0; N 2.07 0 0"

ammonia = """
N  0.000  0.000  0.000; 
H  0.000 -1.772 -0.721; 
H  1.535  0.886 -0.721; 
H -1.535  0.886 -0.721
"""

benzene = """
C        0.598362921      0.000000000     -4.742986733;
C       -0.557705772     -0.354690359     -4.044822733;
C        1.754431614      0.354690359     -4.044822733;
H       -1.457130878     -0.630640582     -4.587995733;
H        2.653856720      0.630640582     -4.587995733;
C       -0.557705772     -0.354690359     -2.648492733;
C        1.754431614      0.354690359     -2.648492733;
H       -1.457130878     -0.630640582     -2.105319733;
H        2.653856720      0.630640582     -2.105319733;
C        0.598362921      0.000000000     -1.950328733;
H        0.598362921      0.000000000     -0.863981733;
H        0.598362921      0.000000000     -5.829333733
"""

test_systems = {
    "H2O": water, 
    "H2": h2_system, 
    "N2": n2_system,
    "NH3": ammonia,
    "C6H6": benzene
}

Our grid-based calculation of $E_{XC}$ and $V_{XC}$:

In [3]:
import numpy
from math import pi

factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0

def custom_eval_xc_LDA(a, p):
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p - 1.0)
        vrho = a * p * rho ** (p - 1.0)
        vxc = (vrho, None, None, None)
        fxc = a * p * (p - 1.0) * rho ** (p - 2.0)  # 2nd order functional derivative
        return exc, vxc, fxc, None
    return eval_xc

At first, reproduce C-implemented LDA with our grid-based LDA:

In [4]:
for system_name, system in test_systems.items():
    print("System name:", system_name)
    mol = mol = gto.M(atom=system, basis="6-31G", unit = 'Bohr', verbose=0)
    calc_c = dft.RKS(mol)
    calc_c.xc = "lda,"
    ene_c = calc_c.kernel()
    print("C-based results:", ene_c)
    
    calc_grid = dft.RKS(mol)
    calc_grid = calc_grid.define_xc_(custom_eval_xc_LDA(factor, power), 'LDA')
    ene_grid = calc_grid.kernel()
    print("Grid-based results:", ene_grid)
    print("Difference:", ene_grid-ene_c, "\n")

System name: H2O
C-based results: -75.154705340357
Grid-based results: -75.15470534035708
Difference: -8.526512829121202e-14 

System name: H2
C-based results: -1.0386177856738832
Grid-based results: -1.0386177856738827
Difference: 4.440892098500626e-16 

System name: N2
C-based results: -107.63947353831051
Grid-based results: -107.63947353831051
Difference: 0.0 

System name: NH3
C-based results: -55.41360440678286
Grid-based results: -55.413604406782774
Difference: 8.526512829121202e-14 

System name: C6H6
C-based results: -213.41468421598614
Grid-based results: -213.41468421592077
Difference: 6.536993168992922e-11 



"Decorator" for derivative calculation code:

In [5]:
def initialize_LDA_derivatives_calculation(calc, a, p):
    def eval_xc_wrt_a(xc_code, rho, *args, **kwargs):
        dexc_wrt_da = rho ** (p - 1.0)
        dvrho_wrt_da = p * rho ** (p - 1.0)
        dvxc_wrt_da = (dvrho_wrt_da, None, None, None)
        return dexc_wrt_da, dvxc_wrt_da, None, None
    def eval_xc_wrt_p(xc_code, rho, *args, **kwargs):
        dexc_wrt_dp = a * numpy.log(rho) * rho ** (p - 1.0)
        dvrho_wrt_dp = a * p * numpy.log(rho) * rho ** (p - 1.0)
        dvxc_wrt_dp = (dvrho_wrt_dp, None, None, None)
        return dexc_wrt_dp, dvxc_wrt_dp, None, None
    calc.number_of_DFT_parameters = 2
    calc.partial_derivative_from_energy_wrt = [eval_xc_wrt_a, eval_xc_wrt_p]

Construct important grid values:

In [6]:
import types

def construct_on_grid_values(calc):
    grid_values = types.SimpleNamespace()

    dm_in_nonorthogonal_basis = calc.make_rdm1()
    grid_values.coords = calc.grids.coords
    grid_values.weights = calc.grids.weights
    non_orthogonal_ao = pyscf.dft.numint.eval_ao(calc.mol, grid_values.coords, deriv=0)
    grid_values.non_orthogonal_ao = non_orthogonal_ao
    grid_values.rho = pyscf.dft.numint.eval_rho(calc.mol, non_orthogonal_ao, 
                                                dm_in_nonorthogonal_basis, xctype='LDA')
    return grid_values

Integrating over the grid:

In [7]:
def calculate_LDA_derivatives(calc):
    list_of_dexc_wrt_dtheta = []
    list_of_dvxc_wrt_dtheta = []
    
    number_of_parameters = calc.number_of_DFT_parameters
    grid_values = construct_on_grid_values(calc)
    
    for p_number in range(number_of_parameters):
        dexc_wrt_dtheta_on_grid = calc.partial_derivative_from_energy_wrt[p_number]('LDA', grid_values.rho)[0]

        current_dexc_wrt_dtheta = numpy.einsum('r,r,r->...', 
                                               dexc_wrt_dtheta_on_grid, 
                                               grid_values.rho, 
                                               grid_values.weights)
        list_of_dexc_wrt_dtheta.append(current_dexc_wrt_dtheta)
    return numpy.array(list_of_dexc_wrt_dtheta)

Finite difference code:

In [8]:
def finite_difference_grid_based(mol, a, p, coeff=0.000001):
    def PySCF_calculation(mol, a, p):
        calc = dft.RKS(mol)
        calc = calc.define_xc_(custom_eval_xc_LDA(a, p), 'LDA')
        ene = calc.kernel()
        return ene
    y = PySCF_calculation(mol, a, p)
    da = a * coeff
    dp = p * coeff
    y_plus_dy_a = PySCF_calculation(mol, a+da, p)
    y_plus_dy_p = PySCF_calculation(mol, a, p+dp)
    dy_wrt_da = (y_plus_dy_a - y) / da
    dy_wrt_dp = (y_plus_dy_p - y) / dp
    return numpy.array([dy_wrt_da, dy_wrt_dp])

In [9]:
for system_name, system in test_systems.items():
    print("System name:", system_name)
    mol = gto.M(atom=system, basis="6-31G", unit = 'Bohr', verbose=0)

    findiff_der = finite_difference_grid_based(mol, factor, power)
    print("Fin diff:", findiff_der)
    
    calc_c = dft.RKS(mol)
    calc_c.xc = "lda,"
    calc_c.kernel()
    initialize_LDA_derivatives_calculation(calc_c, factor, power)
    c_der = calculate_LDA_derivatives(calc_c)
    print("C-based results:", c_der)
    print("Difference with fin diff:", c_der-findiff_der)
    
    calc_grid = dft.RKS(mol)
    calc_grid = calc_grid.define_xc_(custom_eval_xc_LDA(factor, power), 'LDA')
    calc_grid.kernel()
    initialize_LDA_derivatives_calculation(calc_grid, factor, power)
    grid_der = calculate_LDA_derivatives(calc_grid)
    print("Grid-based results:", grid_der)
    print("Difference with fin diff:", grid_der-findiff_der, "\n")

System name: H2O
Fin diff: [ 10.94899416 -11.08017371]
C-based results: [ 10.94899516 -11.08012795]
Difference with fin diff: [9.99385783e-07 4.57537188e-05]
Grid-based results: [ 10.94899516 -11.08012795]
Difference with fin diff: [9.99385790e-07 4.57537188e-05] 

System name: H2
Fin diff: [0.7483585  1.47153018]
C-based results: [0.74835848 1.47153331]
Difference with fin diff: [-2.09378173e-08  3.12685638e-06]
Grid-based results: [0.74835848 1.47153331]
Difference with fin diff: [-2.09378175e-08  3.12685638e-06] 

System name: N2
Fin diff: [ 16.01839344 -17.00558817]
C-based results: [ 16.01839303 -17.00552334]
Difference with fin diff: [-4.12951362e-07  6.48343771e-05]
Grid-based results: [ 16.01839303 -17.00552334]
Difference with fin diff: [-4.12951330e-07  6.48343772e-05] 

System name: NH3
Fin diff: [ 9.33544906 -6.24776715]
C-based results: [ 9.3354491  -6.24773122]
Difference with fin diff: [4.09435525e-08 3.59270047e-05]
Grid-based results: [ 9.3354491  -6.24773122]
Differen